In [1]:
import nltk

# Download required NLTK data (run once)
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')   # often needed with wordnet

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

import json
import pickle
import tensorflow

[nltk_data] Downloading package punkt to /home/mrityunjay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/mrityunjay/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/mrityunjay/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
2025-12-04 18:56:01.856819: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-04 18:56:01.882132: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-04 18:56:01.882155: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-04 18:56:01.883088: E

In [2]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout , Activation, Flatten , Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD
import random

In [3]:
words=[]
labels = []
docs = []
ignore_list = ['?', '!']

In [4]:
dataset = open('intents.json').read()
intents = json.loads(dataset)

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        word_token = nltk.word_tokenize(pattern)
        words.extend(word_token)
        #add documents in the corpus
        docs.append((word_token, intent['tag']))

        # add to our labels list
        if intent['tag'] not in labels:
            labels.append(intent['tag'])

In [6]:
# lemmatize each word, and sort words by removing duplicates:
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_list]
words = sorted(list(set(words)))

In [7]:
# sort labels:
labels = sorted(list(set(labels)))

In [8]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(labels,open('labels.pkl','wb'))

In [9]:
training_data = []
output = [0] * len(labels)

for doc in docs:
    bag_of_words = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    for w in words:
        bag_of_words.append(1 if w in pattern_words else 0)
            
    output_row = list(output)
    output_row[labels.index(doc[1])] = 1
    
    training_data.append([bag_of_words, output_row])

In [10]:
# convert training_data to numpy array and shuffle the data:
random.shuffle(training_data)
# split into features and labels
x_train = []
y_train = []

for bag_of_words, output_row in training_data:
    x_train.append(bag_of_words)
    y_train.append(output_row)

x_train = np.array(x_train)
y_train = np.array(y_train)

In [16]:
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)

x_train shape: (47, 88)
y_train shape: (47, 9)


In [13]:
# Creating Model:

model = Sequential()
model.add(Dense(128, input_shape=(len(x_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))

2025-12-04 18:58:27.404796: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-04 18:58:27.408691: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               11392     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 9)                 585       
                                                                 
Total params: 20233 (79.04 KB)
Trainable params: 20233 (79.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
sgd_optimizer = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd_optimizer, metrics=['accuracy'])

In [19]:
# fit the model 
history = model.fit(np.array(x_train), np.array(y_train), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
10/10 [==============================] - 0s 1ms/step - loss: 2.1748 - accuracy: 0.1702
Epoch 2/200
10/10 [==============================] - 0s 736us/step - loss: 2.1756 - accuracy: 0.2128
Epoch 3/200
10/10 [==============================] - 0s 712us/step - loss: 2.0810 - accuracy: 0.2553
Epoch 4/200
10/10 [==============================] - 0s 806us/step - loss: 1.9956 - accuracy: 0.4043
Epoch 5/200
10/10 [==============================] - 0s 809us/step - loss: 1.8870 - accuracy: 0.4894
Epoch 6/200
10/10 [==============================] - 0s 753us/step - loss: 1.8612 - accuracy: 0.3404
Epoch 7/200
10/10 [==============================] - 0s 795us/step - loss: 1.6268 - accuracy: 0.5106
Epoch 8/200
10/10 [==============================] - 0s 727us/step - loss: 1.6084 - accuracy: 0.4894
Epoch 9/200
10/10 [==============================] - 0s 732us/step - loss: 1.3857 - accuracy: 0.6596
Epoch 10/200
10/10 [==============================] - 0s 815us/step - loss: 1.2896 - accuracy

In [20]:
model.save('chatbot_Application_model.h5', history)

/home/mrityunjay/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
